In [1]:
#Reading the data
import cv2 #(OpenCV)
import numpy as np
import dlib

# Face Detection

In [2]:
#Using HAAR Cascade Classifier
face_cf = cv2.CascadeClassifier('C:\\Computer_vision\\opencv\\data\\haarcascades\\haarcascade_frontalface_default.xml')

#Load the image
img = cv2.imread('C:\\Computer_vision\\Vk.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

faces = face_cf.detectMultiScale(gray,1.3,5)
print(type(faces))

if faces is ():
  print("No faces found in the given image.")

for x,y,w,h in faces:
  cv2.rectangle(img,(x,y),(x+w,y+h),(112,0,0),2)
  cv2.imshow('Face Detection',img) #--- This line of code is depreciated in google colab as it's facing issues crashing the jupyter Notebook
#   cv2_imshow(img)
  cv2.waitKey(0)

cv2.destroyAllWindows()

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?


<class 'numpy.ndarray'>


C:\Users\MI\AppData\Local\Temp\ipykernel_10616\2841309542.py:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


# Combined Face and Eye Detector

In [3]:
#Using HAAR Cascade Classifier
face_cf = cv2.CascadeClassifier('C:\\Computer_vision\\opencv\\data\\haarcascades\\haarcascade_frontalface_default.xml')

eye_cf = cv2.CascadeClassifier('C:\\Computer_vision\\opencv\\data\\haarcascades\\haarcascade_eye.xml')

#Load the image
img = cv2.imread('C:\\Computer_vision\\Vk.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

faces = face_cf.detectMultiScale(gray,1.3,5)

if faces is ():
  print("No faces found in the given image.")

for x,y,w,h in faces:
  cv2.rectangle(img,(x,y),(x+w,y+h),(112,0,0),2)
  cv2.imshow('Face Detection',img) #--- This line of code is depreciated in google colab as it's facing issues crashing the jupyter Notebook
  # cv2_imshow(img)
  # cv2.waitKey(1500)
  roi_gray = gray[y:y+h,x:x+w]
  roi_color = img[y:y+h,x:x+w]
  
  eyes = eye_cf.detectMultiScale(roi_gray,1.1,3)
  #Eye capturing
  for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,180,0),2)
    cv2.imshow('Face and eye',img)
    # cv2_imshow(img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\MI\AppData\Local\Temp\ipykernel_10616\2767679419.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


# Detect Face and Eye from the video (Webcam)

In [4]:
def face_eye_dt(gray,img):
  faces = face_cf.detectMultiScale(gray,1.3,5)
  for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(118,0,0),2)
    roi_gray = gray[y:y+h,x:x+w]
    roi_img = img[y:y+h,x:x+w]

    eyes = eye_cf.detectMultiScale(roi_gray,1.1,3)
    for (ex,ey,ew,eh) in eyes:
      cv2.rectangle(roi_img,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    return img

In [8]:
#Video initialization
video_cpt = cv2.VideoCapture(0) # 0 --> To user inbuilt webcam, 1 --> To use external webcam

while True:
  ret,frame = video_cpt.read()
  #If frame is read correctly ret is True
  if ret is not True:
    print("Can't read the frame, Exiting...")
    break
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  live_vd = face_eye_dt(gray,frame)
  cv2.imshow('Video',live_vd)
  if cv2.waitKey(0) or ord('q'):
    break

video_cpt.release()
cv2.destroyAllWindows()

# Detect the center of pupil and track its movement

In [9]:
cap = cv2.VideoCapture('eye_motion_tracking\eye_recording.flv')

while True:
    ret,frame = cap.read()
    if ret is not True:
        print("Can't read the frame, Exiting...")
        break

    roi = frame[269:795,537:1416]
    rows,cols,clr = roi.shape
    gray_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray_roi_blur = cv2.GaussianBlur(gray_roi,(7,7),0) #(7,7) is the kernel size (more is the number more blur)
    

    _,threshold = cv2.threshold(gray_roi_blur,3,255,cv2.THRESH_BINARY_INV)
    contours,heirr = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # print(contours)

    sorted_contours = sorted(contours, key = lambda x: cv2.contourArea(x), reverse=True)
    for i in sorted_contours:
        (x,y,w,h) = cv2.boundingRect(i)
        cv2.rectangle(roi,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.line(roi,(x+int(w/2),0),(x+int(w/2),rows),(0,255,0),2) #Horizontal line from the center of pupil
        cv2.line(roi,(0,y+int(h/2)),(cols,y+int((h/2))),(0,255,0),2) #Vertical line from center of pupil
        break


    cv2.imshow('Gray roi',gray_roi)
    cv2.imshow('Blur gray roi',gray_roi_blur)
    cv2.imshow('threshold',threshold)
    cv2.imshow('ROI',roi)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Can't read the frame, Exiting...


# Trying out with webcam


In [10]:
import cv2
import dlib
from math import hypot
import numpy as np

In [11]:
vd = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    ret,frame = vd.read()
    new_frame = np.zeros((500,500,3),np.uint8)
    if ret is not True:
        print("Cant read the frame, Exiting...")
        break

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    font = cv2.FONT_HERSHEY_COMPLEX

    def mid_point(p1,p2):
        return int((p1.x+p2.x)/2), int((p1.y+p2.y)/2)
    
    #To detect the Blinking (by taking the ratio to horizontal and vertical line)
    
    def get_blink_ratio(eye_points,facial_landmarks):
        #Left or Right Eye points
        left_point = (facial_landmarks.part(eye_points[0]).x,facial_landmarks.part(eye_points[0]).y)
        right_point = (facial_landmarks.part(eye_points[3]).x,facial_landmarks.part(eye_points[3]).y)
        top_point = (mid_point(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[2])))
        bottom_point = (mid_point(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[4])))

        #lines
        hor_line = cv2.line(frame,left_point,right_point,(0,122,0),2)
        ver_line = cv2.line(frame,top_point,bottom_point,(0,122,0),2)

        #Distance
        hor_length = hypot((left_point[0]-right_point[0]),(left_point[1]-right_point[1]))
        ver_length = hypot((top_point[0]-bottom_point[0]),(top_point[1]-bottom_point[1]))

        #Ratio
        blink_ratio = hor_length/ver_length

        return blink_ratio
    
    #To Track the eye movement (Gaze Ratio)
    def gaze_ratio(eye_points,facial_landmarks):
        eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
        
        height,width,_ = frame.shape
        mask = np.zeros((height,width),np.uint8)
        cv2.polylines(mask,[eye_region],True,255,2)
        cv2.fillPoly(mask,[eye_region],255)
        eye = cv2.bitwise_and(gray,gray,mask=mask)

        min_x = np.min(eye_region[:,0])
        max_x = np.max(eye_region[:,0])
        min_y = np.min(eye_region[:,1])
        max_y = np.max(eye_region[:,1])

        gray_eye = eye[min_y:max_y,min_x:max_x]

        #THreshold
        _,threshold_eye = cv2.threshold(gray_eye,70,255,cv2.THRESH_BINARY)
        ht,wt = threshold_eye.shape
        total_px = ht*wt

        #left side white
        left_side_threshold = threshold_eye[0:ht,0:int(wt/2)]
        left_side_white =  cv2.countNonZero(left_side_threshold)

        # r = total_px - left_side_white

        #right side white
        right_side_threshold = threshold_eye[0:ht,int(wt/2):wt]
        right_side_white = cv2.countNonZero(right_side_threshold)

        # l = total_px - right_side_white


        # if left_side_white == 0:
        #     gazeratio = 0.9
        if right_side_white == 0:
            gazeratio = 3
        else:
            gazeratio = left_side_white/right_side_white
        return gazeratio
    
    for face in faces:
        # x,y = face.left(),face.top()
        # x1,y1 = face.right(),face.bottom()

        # cv2.rectangle(frame,(x,y),(x1,y1),(0,122,0),2)

        landmarks = predictor(gray,face)

        #Taking avg of left and right eye blink ratio
        left_eye_blink = get_blink_ratio([36,37,38,39,40,41],landmarks)
        right_eye_blink = get_blink_ratio([42,43,44,45,46,47],landmarks)

        avg_blink_ratio = (left_eye_blink+right_eye_blink)/2

        if avg_blink_ratio > 4.6:
            cv2.putText(frame,"Blinking",(60,90),font,2,(0,0,122))

        #Left eye
        left_point = (landmarks.part(36).x,landmarks.part(36).y)
        right_point = (landmarks.part(39).x,landmarks.part(39).y)
        top_point = (mid_point(landmarks.part(37),landmarks.part(38)))
        bottom_point = (mid_point(landmarks.part(41),landmarks.part(40)))
        
        #lines
        hor_line = cv2.line(frame,left_point,right_point,(0,122,0),2)
        ver_line = cv2.line(frame,top_point,bottom_point,(0,122,0),2)

        #To track the gaze ratio as the avg of both left and right eye
        gaze_ratio_left_eye = gaze_ratio([36,37,38,39,40,41],landmarks)
        gaze_ratio_right_eye = gaze_ratio([42,43,44,45,46,47],landmarks)

        avg_gaze_ratio = (gaze_ratio_left_eye+gaze_ratio_right_eye)/2
        # avg_gaze_ratio = (gaze_ratio_left_eye/gaze_ratio_right_eye)

        # r,l = gaze_ratio([36,37,38,39,40,41],landmarks)

        # if r>l:
        #     cv2.putText(frame,"Looking Left",(20,40),font,2,(0,0,220),2)
        #     new_frame[:] = (0,0,220)
        # elif l>r:
        #     cv2.putText(frame,"Looking Right",(20,40),font,2,(0,0,220),2)
        #     new_frame[:] = (220,0,0)
        # else:
        #     cv2.putText(frame,"Center",(20,40),font,2,(0,0,220),2)

        if avg_gaze_ratio < 0.6:
            cv2.putText(frame,"Looking Right",(20,40),font,2,(0,0,220),2)
            new_frame[:] = (0,0,220)
        elif (0.6<=avg_gaze_ratio<=1.5):
            cv2.putText(frame,"Center",(20,40),font,2,(0,0,220),2)
        else:
            cv2.putText(frame,"Looking Left",(20,40),font,2,(0,0,220),2)
            new_frame[:] = (220,0,0)

    cv2.imshow("frame",frame)
    cv2.imshow("new_frame",new_frame)
    

    key = cv2.waitKey(1)

    if key == 27:
        break

vd.release()
cv2.destroyAllWindows()

In [59]:
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    if ret is not True:
        print("Can't read the frame, Exiting...")
        break

    # roi = frame[269:795,537:1416]
    roi = frame[100:1000,200:1000]
    rows,cols,clr = roi.shape
    gray_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray_roi_blur = cv2.GaussianBlur(gray_roi,(7,7),0) #(7,7) is the kernel size (more is the number more blur)
    

    _,threshold = cv2.threshold(gray_roi_blur,3,255,cv2.THRESH_BINARY_INV)
    contours,heirr = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # print(contours)

    sorted_contours = sorted(contours, key = lambda x: cv2.contourArea(x), reverse=True)
    for i in sorted_contours:
        (x,y,w,h) = cv2.boundingRect(i)
        cv2.rectangle(roi,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.line(roi,(x+int(w/2),0),(x+int(w/2),rows),(0,255,0),2) #Horizontal line from the center of pupil
        cv2.line(roi,(0,y+int(h/2)),(cols,y+int((h/2))),(0,255,0),2) #Vertical line from center of pupil
        break


    # cv2.imshow('Gray roi',gray_roi)
    # cv2.imshow('Blur gray roi',gray_roi_blur)
    # cv2.imshow('threshold',threshold)
    cv2.imshow('ROI',roi)
    cv2.waitKey(0)
    # key = cv2.waitKey(30)
    # if key == 27:
    #     break
    # if cv2.waitKey(0) or ord('q'):
    #     break

cv2.destroyAllWindows()

KeyboardInterrupt: 

In [2]:
import cv2

In [6]:
vd = cv2.VideoCapture(0)
while True:
    ret,img = vd.read()
    if ret is not True:
        print("No frame detected, Exit")
        break

    cv2.imshow("Face Detect",img)
    # if cv2.waitKey(1) or 0xFF == ('q'):
    #     break
    if cv2.waitKey(1) & 0xFF == ('q'):
        break

vd.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

In [ ]:
for cnt in sorted_contours:
    # (x,y,w,h) = cv2.drawContours(roi,[cnt],-1,(0,0,255),2) #--> To draw the contours
    (x,y,w,h) = cv2.boundingRect(cnt)
    cv2.rectangle(roi,(x,y),(x+w,y+h),(0,0,255),2)

    cv2.imshow('roi',roi)
    cv2.waitKey(0)

cv2.destroyAllWindows()    

In [ ]:
#Video initialization
video_cpt = cv2.VideoCapture(0)

while True:
  ret,frame = video_cpt.read()
  #If frame is read correctly ret is True
  if ret is not True:
    print("Can't read the frame, Exiting...")
    break
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  live_vd = face_eye_dt(gray,frame)
  cv2.imshow('Video',live_vd)
  if cv2.waitKey(0) or ord('q'):
    break

video_cpt.release()
cv2.destroyAllWindows()